In [14]:

def div_d(my_dict,sum_p):

    #sum_p = sum(my_dict.values())

    for i in my_dict:
        my_dict[i] = float(my_dict[i]/sum_p)

    return my_dict 

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def fetch_data(exp_fname='TCGA-PAAD(non_norm_exp_data).csv',gene_file='tcga_gene.csv',meta_file='TCGA_meta_info.csv'):
#     exp_fname=raw_in+'TCGA-PAAD(non_norm_exp_data).csv'
#     meta_file=raw_in+'TCGA_meta_info.csv'
#     gene_file=raw_in+'tcga_gene.csv'

    GSE162694_norm=pd.read_csv(raw_in+exp_fname,index_col=0)
    GSE162694_norm.rename(columns={'Row.names':'Patient'},inplace=True)

    keys_index=pd.read_csv(base_in+gene_file)
 
    GSE162694_groups=pd.read_csv(raw_in+meta_file)
    GSE162694_groups['Patient']=GSE162694_groups['Patient'].str.replace('-01A','')
  
    GSE162694_groups=GSE162694_groups[['Patient','Label']]
   
    maped_keys=intersection(keys_index['Genes'].to_list(),GSE162694_norm.columns.tolist())
    GSE162694_sub=GSE162694_norm[['Patient']+maped_keys].copy()

    dataset=pd.merge(GSE162694_sub,GSE162694_groups,on='Patient',   how='right')
    df=dataset.copy()
    ready_data=df.loc[:,~df.columns.duplicated()]
    #ready_data.drop('Patient',inplace=True)
    return ready_data


def feature_extract(model):
    feature_importances=[]
    importance = model.ranking_
    feature_importances = pd.Series(importance, index=model.feature_names_in_)
    sel_indx=model.support_
    my_feat=feature_importances[sel_indx]
    return my_feat

def feature_rank(model):
    feature_importances=[]
    importance = model.feature_importances_
    #feature_importances = pd.Series(importance, index=model.feature_names_in_)
    feature_importances = pd.DataFrame({'Name':model.feature_names_in_,'Score':importance})
    #sel_indx=model.support_
    my_feat=feature_importances.copy()
    return my_feat

In [2]:
# A function to fetch the differnt models in pipeline while comparision
#Parameters should be tuned by user
def get_models():
    models = dict()
    # lr
    #rfe = RFECV(estimator=DecisionTreeClassifier(random_state=24),step=10,cv=10,n_jobs=25)
    model = DecisionTreeClassifier(random_state=24)
    models['dt'] = model
    # perceptron
    #rfe = RFECV(estimator=RandomForestClassifier(random_state=24,n_estimators=50),cv=10,n_jobs=25)
    model = RandomForestClassifier(random_state=24,n_estimators=50)
    models['rf'] = model
    # cart
    #KNN
    #rfe = RFECV(estimator=Perceptron(),cv=10,n_jobs=25)
    model = KNeighborsClassifier()
    models['knn'] = model
    # rf
    #rfe = RFECV(estimator=SVC(kernel='linear',random_state=24),cv=10,n_jobs=25)
    model = SVC(kernel='linear',random_state=24)
    models['svm'] = model
    # gbm
    #rfe = RFECV(estimator=XGBClassifier(random_state=24),cv=10,n_jobs=25)
    model = XGBClassifier(random_state=24)
    models['xgb'] = model
    #Some liner
    #rfe = RFECV(estimator=LogisticRegression(solver='lbfgs',max_iter=1000,random_state=24),cv=10,n_jobs=25)
    model = LogisticRegression(solver='lbfgs',max_iter=1000,random_state=24)
    models['logit'] = model
    return models

In [22]:
# explore the algorithm wrapped by RFE
from numpy import mean
from numpy import std

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from matplotlib import pyplot
from tqdm import tqdm
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

from collections import Counter

le = preprocessing.LabelEncoder()
le.fit(["CP", "NO_CP"])

LabelEncoder()

In [3]:
# the libraries we need

#Models

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC


#Metrics
from sklearn.inspection import permutation_importance
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import tree
from sklearn import preprocessing




#Utility
from sklearn.model_selection import train_test_split
#from imblearn.over_sampling import SMOTE
from collections import Counter
from tqdm import tqdm
from glob import glob

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import numpy as np

#import imblearn
import time





In [4]:
le = preprocessing.LabelEncoder()
le.fit(["CP", "NO_CP"])

raw_in='../Input/raw/'
base_in='../Input/Prep/Script1/'
base_out='../Output/Script1/Results/'
glob(raw_in+'*')

['../Input/raw\\Data description.docx',
 '../Input/raw\\dump',
 '../Input/raw\\PAAD-Methylation(CpG).cct',
 '../Input/raw\\PAAD-Mutation(Site).txt',
 '../Input/raw\\TCGA-PAAD(non_norm_exp_data).csv',
 '../Input/raw\\tcga_gene.csv',
 '../Input/raw\\TCGA_meta_info.csv']

## Dataset preparation

In [23]:

# define dataset
#X, y = get_dataset()
#gene_list='FDR_list_cutoff.05.xlsx'
#output_file='FDR_full_compare.xlsx'
#gene_list='DADA.xlsx'

#Name of outfile to store the comparison results
output_file=base_out+'full_compare.xlsx'
# This is a custom data generation function
dataset=fetch_data()#Replace it with your dataframe containg last column as Label
display(dataset.shape,dataset.head())

X=dataset.iloc[:,1:-1]
labels=dataset.iloc[:,-1]
y=le.transform(labels)
# get the models to evaluate

        


(110, 20026)

,Patient,BCAR1,CXCL6,MIPOL1,EOMES,HS3ST6,BMP8B,ITPRIP,EIF5AL1,C17orf54,...,SCARNA9,ECM2,LRRN4CL,APCDD1,MPG,PGBD2,ZNF414,C19orf66,NCRNA00201,Label
0,TCGA-2J-AAB9,5279.758452,26886.562002,6073.553398,2648.799322,0.000000,6485.317937,4613.460070,6324.375426,2199.41522,...,1370.819977,4342.729718,3804.895583,5339.303014,5012.976707,4909.203848,6677.272542,4307.424281,2771.992135,CP
1,TCGA-2J-AABF,4515.108156,19987.272231,4426.668282,8017.097250,0.000000,3030.328371,3038.965542,5216.185563,0.00000,...,5439.154803,4842.467644,1845.172649,4512.333226,4041.429017,9184.648805,5233.333082,6585.777360,7183.037226,CP
2,TCGA-2J-AABH,6385.855593,7562.995790,3715.330413,1407.560192,1229.571259,4529.043072,3317.775922,5667.000259,0.00000,...,3342.253000,2403.651948,968.922840,4061.283565,6106.826478,5652.210190,5024.755652,4404.892661,6903.351359,CP
3,TCGA-2J-AABO,4618.793055,157.278863,6249.377128,1980.245301,0.000000,17100.039808,3184.287190,5197.879970,0.00000,...,4099.286350,3649.634496,6333.805780,9507.169690,5092.178066,7584.873270,3651.485110,5192.400220,4367.192122,CP
4,TCGA-2J-AABP,4701.518714,337.294017,1602.612340,10200.163378,0.000000,1944.082681,13550.299019,18552.251010,0.00000,...,4709.619708,3601.309493,7394.985123,1125.774048,4241.510739,3473.562857,6250.891095,2940.032432,1210.600421,CP


## Repeated 10 times 10-fold cross validation criteria for model comparision

In [6]:
models = get_models()

results={}
with pd.ExcelWriter(output_file) as writer:


    for name, model in tqdm(models.items()):

        rkf=RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
        cross_out = cross_validate(model, X, y,return_train_score=True,return_estimator=True ,scoring=('accuracy','f1','roc_auc','precision','recall'),n_jobs=24 ,cv=rkf)
        results[name]=cross_out
        result_df=pd.DataFrame(cross_out)
        result_df.to_excel(writer, sheet_name=name)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:57<00:00, 39.62s/it]


# Result visualizatoin for the six algorithms

In [7]:
#Creating a copy
FDR_res=results.copy()

In [9]:
#FDR_res,PDR_old_res,neg_res
my_df=pd.DataFrame([])
for name,my_entry in FDR_res.items():
    df=pd.DataFrame(my_entry)
    df['Algo']=name
    my_df=my_df.append(df)
    
my_df.drop(['fit_time','score_time','estimator'],axis=1,inplace=True)
with pd.ExcelWriter('Algo_compare_full_feat.xlsx') as writer:# Own file name can be used
    
    df_mean=my_df.groupby('Algo').mean()
    df_std=my_df.groupby('Algo').std()
    df_mean.to_excel(writer,sheet_name='mean')
    df_std.to_excel(writer,sheet_name='std')
#my_df.head()
fig = px.box(my_df,color='Algo',title='New Gene List')
#fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()